# Image Search Engine Demo

This notebook demonstrates how to use the image search engine interactively.

In [ ]:
import sys
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image

# Add src to path
sys.path.insert(0, str(Path.cwd().parent))

from src import ImageSearchEngine, FeatureExtractor
from src.utils import load_image_paths

## 1. Initialize the Search Engine

In [ ]:
# Initialize with ResNet50 (default)
engine = ImageSearchEngine(
    model_name="resnet50",
    index_type="cosine"
)

print("✓ Search engine initialized")

## 2. Build or Load Index

In [ ]:
# Path to your images
image_dir = Path("../data/raw")
index_dir = Path("../data/index")

# Check if index exists
if index_dir.exists() and (index_dir / "index.faiss").exists():
    print("Loading existing index...")
    engine.load(index_dir)
else:
    print("Building new index...")
    engine.build_index(
        image_dir=image_dir,
        batch_size=32,
        save_path=index_dir
    )

print(f"✓ Index ready with {len(engine.image_paths)} images")

## 3. Perform a Search

In [ ]:
# Get available images
image_paths = load_image_paths(image_dir)

if len(image_paths) > 0:
    # Use the first image as query
    query_image_path = image_paths[0]
    print(f"Query image: {query_image_path.name}")
    
    # Search for similar images
    results = engine.search(query_image_path, top_k=10)
    
    # Display results
    for result in results[:5]:  # Show top 5
        print(f"  {result['rank']}. {Path(result['path']).name} - Score: {result['score']:.4f}")
else:
    print("No images found. Please add images to data/raw/")

## 4. Visualize Results

In [ ]:
def visualize_search_results(query_path, results, top_k=5):
    """
    Visualize query image and top search results.
    """
    fig, axes = plt.subplots(1, top_k + 1, figsize=(3 * (top_k + 1), 3))
    
    # Show query
    query_img = Image.open(query_path)
    axes[0].imshow(query_img)
    axes[0].set_title("Query", fontsize=14, fontweight='bold')
    axes[0].axis('off')
    
    # Show results
    for idx, result in enumerate(results[:top_k], 1):
        img = Image.open(result['path'])
        axes[idx].imshow(img)
        axes[idx].set_title(
            f"#{result['rank']}\n{result['score']:.3f}",
            fontsize=11
        )
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()

# Visualize if we have results
if len(image_paths) > 0:
    visualize_search_results(query_image_path, results, top_k=5)

## 5. Try Different Query Images

In [ ]:
# Try searching with different images
if len(image_paths) > 1:
    query_idx = 1  # Change this to try different images
    query_path = image_paths[query_idx]
    
    print(f"Searching with: {query_path.name}")
    results = engine.search(query_path, top_k=10)
    
    visualize_search_results(query_path, results, top_k=5)

## 6. Experiment with Different Models

In [ ]:
# Try a different model
# Options: resnet50, efficientnet_b0, vit_base_patch16_224, convnext_tiny, etc.

engine_v2 = ImageSearchEngine(
    model_name="efficientnet_b0",
    index_type="cosine"
)

print("Building index with EfficientNet...")
if len(image_paths) > 0:
    engine_v2.build_index(image_dir, batch_size=32)
    print("✓ Index built")

## 7. Compare Results Between Models

In [ ]:
if len(image_paths) > 0:
    query = image_paths[0]
    
    # Search with both models
    results_resnet = engine.search(query, top_k=5)
    results_efficient = engine_v2.search(query, top_k=5)
    
    print("ResNet50 Results:")
    for r in results_resnet:
        print(f"  {r['rank']}. {Path(r['path']).name} - {r['score']:.4f}")
    
    print("\nEfficientNet Results:")
    for r in results_efficient:
        print(f"  {r['rank']}. {Path(r['path']).name} - {r['score']:.4f}")